# Requête avec paramètres

L'endpoint  http://api.open-notify.org/astros.json utilisé précédemment n'exigeait pas de paramètres - une simple requête GET permettait d'obtenir le nombre de personnes actuellement dans l'espace.

Il est plus fréquent que l'API requière des paramètres, pour préciser la requête. Utilisons l'endpoint http://api.open-notify.org/iss-pass.json. La documentation nous apprend que cet endpoint a besoin en entrée des paramètres **lat (latitude) et long (longitude)**, pour donner en sortie **les horaires des prochains passages de l'ISS au-dessus d'un point donné**.

Les paramètres **doivent être fournis sous forme de dictionnaire**. `requests` les convertira automatiquement en JSON pour les insérer dans l'URL de requête.

In [1]:
import requests

In [2]:
# syntaxe des paramètres
parameters = {
              "lat": 48.881,
              "lon": 2.362
             }

In [3]:
# requête incluant les paramètres
response = requests.get("http://api.open-notify.org/iss-pass.json",
                        params=parameters)

In [4]:
response.status_code

404

In [6]:
# Si on ne remplit pas les paramètres : 
r = requests.get("http://api.open-notify.org/iss-pass.json")
print(r.status_code)
r.json()

404


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

BONUS:  La requête créée et envoyée par le package `requests` pourrait être écrite entièrement "à la main", en insérant les **paramètres directement dans l'URL** (risque d'erreur) :

In [6]:
requests.get("http://api.open-notify.org/iss-pass.json?lat=48.881&lon=2.362") 
# Exécutable également dans le navigateur

<Response [200]>

In [7]:
# Méthode pour afficher l'URL complet d'une requête paramétrée
req = requests.Request('GET', "http://api.open-notify.org/iss-pass.json", params=parameters)
prep = req.prepare()
print(prep.url)

http://api.open-notify.org/iss-pass.json?lat=48.881&lon=2.362


Analysons le contenu de la réponse :

In [8]:
response.content

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1631274932, \n    "latitude": 48.881, \n    "longitude": 2.362, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 322, \n      "risetime": 1631305048\n    }, \n    {\n      "duration": 617, \n      "risetime": 1631310652\n    }, \n    {\n      "duration": 657, \n      "risetime": 1631316428\n    }, \n    {\n      "duration": 653, \n      "risetime": 1631322253\n    }, \n    {\n      "duration": 657, \n      "risetime": 1631328071\n    }\n  ]\n}\n'

In [9]:
# réponse sous forme de dictionnaire
response.json()

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1631274932,
  'latitude': 48.881,
  'longitude': 2.362,
  'passes': 5},
 'response': [{'duration': 322, 'risetime': 1631305048},
  {'duration': 617, 'risetime': 1631310652},
  {'duration': 657, 'risetime': 1631316428},
  {'duration': 653, 'risetime': 1631322253},
  {'duration': 657, 'risetime': 1631328071}]}

La réponse JSON consiste en un dictionnaire contenant 3 clés. la troisème est celle qui nous intéresse, et est elle-même un dictionnaire contenant le **risetime** (début d'un passage de l'ISS) et la **durée de visibilité**.

Les `risetime` sont au format **timestamp**, qui correspond par convention au nombre de secondes depuis le 01/01/1970. Pour rendre ces nombres plus compréhensibles, il est possible de les **convertir en format datetime** avec la méthode `datetime.fromtimestamp()`.

In [10]:
passages = response.json()['response']
passages

[{'duration': 322, 'risetime': 1631305048},
 {'duration': 617, 'risetime': 1631310652},
 {'duration': 657, 'risetime': 1631316428},
 {'duration': 653, 'risetime': 1631322253},
 {'duration': 657, 'risetime': 1631328071}]

In [11]:
risetimes = [ iss_pass['risetime'] for iss_pass in passages ]
risetimes

[1631305048, 1631310652, 1631316428, 1631322253, 1631328071]

In [14]:
# M&thode sans liste en compréhension 
risetimes = []
for iss_pass in passages:
    risetimes.append(iss_pass['risetime'])
risetimes

[1631305048, 1631310652, 1631316428, 1631322253, 1631328071]

In [15]:
from datetime import datetime

correct_times = []

for t in risetimes:
    correct_time = datetime.fromtimestamp(t)
    correct_times.append(correct_time)
    print(correct_time)

2021-09-10 22:17:28
2021-09-10 23:50:52
2021-09-11 01:27:08
2021-09-11 03:04:13
2021-09-11 04:41:11


In [ ]:
# Méthode avec liste en compréhension
correct_times = [ datetime.fromtimestamp(t) for t in risetimes ]
correct_times

L'ISS passe souvent dans la zone au-dessus du point passé en paramètres - quasiment 1 fois toutes les heure et demie !